In [ ]:
! ./udpipe-1.2.0 --tokenize --tag --parse bosque-ud-2.6.udpipe dhbb.raw > dhbb.conllu
! ./udpipe-1.2.0 --tokenize --tag --parse bosque-ud-2.6.udpipe obras.raw > obras.conllu

In [1]:
import resource
gb = 2**30
resource.setrlimit(resource.RLIMIT_AS, (10*gb,10*gb))

In [2]:
# Definindo variáveis iniciais
import estrutura_ud
import os

root_deprel = ["root"]
subordinate_deprel = ["acl:relcl", "ccomp", "advcl"]
max_examples = 20

path = {
    'bosque': "bosque-ud-2.6.conllu",
    'dhbb': "dhbb.conllu",
    "obras": "obras.conllu"
}

In [ ]:
# Carregamento dos corpora (demora muito!)
bosque = estrutura_ud.Corpus(recursivo=True)
dhbb = estrutura_ud.Corpus(recursivo=True)
obras = estrutura_ud.Corpus(recursivo=True)
bosque.load(path["bosque"])
print("bosque ok")
dhbb.load(path["dhbb"])
print("dhbb ok")
obras.load(path["obras"])
print("obras ok")

corpora = {
    'dhbb': dhbb,
    'obras': obras,
    'bosque': bosque
}

bosque ok


--- Logging error ---
Traceback (most recent call last):
  File "/home/elvis/desocultando-sujeitos/estrutura_ud.py", line 108, in build
  File "/home/elvis/desocultando-sujeitos/estrutura_ud.py", line 58, in build
MemoryError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/elvis/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
  File "<ipython-input-3-8328003cc172>", line 7, in <module>
  File "/home/elvis/desocultando-sujeitos/estrutura_ud.py", line 219, in load
  File "/home/elvis/desocultando-sujeitos/estrutura_ud.py", line 174, in build
  File "/home/elvis/desocultando-sujeitos/estrutura_ud.py", line 116, in build
TypeError: unsupported operand type(s) for +: 'MemoryError' and 'str'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/elvis/.local/lib/python3.6/site-packages/IPython/core/interactiveshell

In [ ]:
# Estatísticas dos corpora
stats_corpora = {}
for corpus in corpora:
    stats_corpora[corpus] = {
        "size": os.path.getsize(path[corpus])/1024, 
        "n_tokens": 0, 
        "n_sentences": 0,
    }
    
for corpus in corpora:
    for sentence in corpora[corpus].sentences.values():
        stats_corpora[corpus]['n_sentences'] += 1
        for token in sentence.tokens:
            if not '-' in token.id:
                stats_corpora[corpus]['n_tokens'] += 1
    
print(stats_corpora)

In [ ]:
# Números de orações
total_deprel = {}
for corpus in corpora:
    total_deprel[corpus] = {
        "|".join(root_deprel): 0,
        "|".join(subordinate_deprel): 0
    }
    for sentence in corpora[corpus].sentences.values():
        for token in sentence.tokens:
            if token.deprel in root_deprel:
                total_deprel[corpus]["|".join(root_deprel)] += 1
            if token.deprel in subordinate_deprel:
                total_deprel[corpus]["|".join(subordinate_deprel)] += 1
print(total_deprel)

# Busca ingênua tanto em root quanto em subordinada

In [ ]:
def find_clause_without_subject(sentences, deprel_list):
    clauses_without_subject = {}
    for sent_id, sentence in sentences.items():
        for t, token in enumerate(sentence.tokens):
            if token.deprel in deprel_list:
                has_subj = False
                for _token in sentence.tokens:
                    if _token.dephead == token.id and _token.deprel in ["nsubj", "csubj", "nsubj:pass"]:
                        has_subj = True
                        break
                if not has_subj:
                    if not sent_id in clauses_without_subject:
                        clauses_without_subject[sent_id] = []
                    clauses_without_subject[sent_id].append(t)
    return clauses_without_subject

root_without_subject = {}
subordinate_without_subject = {}
for corpus in corpora:
    root_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, root_deprel)
    subordinate_without_subject[corpus] = find_clause_without_subject(corpora[corpus].sentences, subordinate_deprel)

In [ ]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto ingênuo: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break

# Filtro A

In [ ]:
def find_haver_impessoal(sentences, list_tokens):
    haver_impessoal = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma == "haver" and "Number=Sing" in token.feats and "Person=3" in token.feats:
                if not sent_id in haver_impessoal:
                    haver_impessoal[sent_id] = []
                haver_impessoal[sent_id].append(t)
    return haver_impessoal
                
root_haver_impessoal = {}
subordinate_haver_impessoal = {}
for corpus in corpora:
    root_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_haver_impessoal[corpus] = find_haver_impessoal(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_haver_impessoal[corpus]:
        for token in root_haver_impessoal[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_haver_impessoal[corpus]:
        for token in subordinate_haver_impessoal[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [ ]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in root_haver_impessoal[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e haver impessoal: {}".format(corpus, sum([len(x) for x in subordinate_haver_impessoal[corpus].values()])))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de haver impessoal na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_haver_impessoal[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de haver impessoal na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_haver_impessoal[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break

# Filtro B

In [ ]:
def find_nominals(sentences, list_tokens):
    nominals = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.upos != "VERB":
                is_cop = False
                for _token in sentence.tokens:
                    if _token.head_token.id == token.id and _token.deprel == "cop":
                        is_cop = True
                        break
                if not is_cop:
                    if not sent_id in nominals:
                        nominals[sent_id] = []
                    nominals[sent_id].append(t)
    return nominals
                
root_nominals = {}
for corpus in corpora:
    root_nominals[corpus] = find_nominals(corpora[corpus].sentences, root_without_subject[corpus])
    
    for sentence in root_nominals[corpus]:
        for token in root_nominals[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)

In [ ]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e nominal: {}".format(corpus, sum([len(x) for x in root_nominals[corpus].values()])))

for corpus in corpora:
    i = 0
    print("\n=== {} exemplos de nominais como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_nominals[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break

# Filtro C

In [ ]:
fenomenos_natureza = "chover|ventar|nevar|chuviscar|garoar|gear|anoitecer|amanhecer|entardecer|relampejar|trovejar".split("|")
def find_natureza(sentences, list_tokens):
    natureza = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if t in list_tokens[sent_id] and token.lemma in fenomenos_natureza:
                if not sent_id in natureza:
                    natureza[sent_id] = []
                natureza[sent_id].append(t)
    return natureza
                
root_natureza = {}
subordinate_natureza = {}
for corpus in corpora:
    root_natureza[corpus] = find_natureza(corpora[corpus].sentences, root_without_subject[corpus])
    subordinate_natureza[corpus] = find_natureza(corpora[corpus].sentences, subordinate_without_subject[corpus])
    
    for sentence in root_natureza[corpus]:
        for token in root_natureza[corpus][sentence]:
            root_without_subject[corpus][sentence].remove(token)
    for sentence in subordinate_natureza[corpus]:
        for token in subordinate_natureza[corpus][sentence]:
            subordinate_without_subject[corpus][sentence].remove(token)

In [ ]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in root_natureza[corpus].values()])))
    print("{} - orações subordinadas com sujeito oculto e fenômeno natureza: {}".format(corpus, sum([len(x) for x in subordinate_natureza[corpus].values()])))
    
for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de fenômeno natureza como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_natureza[corpus].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de fenômeno natureza como oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_natureza[corpus].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break

# Filtro D

In [ ]:
def find_se(sentences, list_tokens):
    se = {"1": {}, "2": {}, "3": {}}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and "Gender=Unsp" in token.feats and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                if not sent_id in se["1"]:
                    se["1"][sent_id] = []
                se["1"][sent_id].append(sentence.map_token_id[token.head_token.id])
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and "VerbForm=Inf" in token.head_token.feats and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                if not sent_id in se["2"]:
                    se["2"][sent_id] = []
                se["2"][sent_id].append(sentence.map_token_id[token.head_token.id])
            if token.head_token.id != "_" and token.lemma == "se" and token.upos == "PRON" and sentence.map_token_id[token.head_token.id] in list_tokens[sent_id]:
                for _token in sentence.tokens:
                    if _token.deprel == "case" and _token.head_token.head_token.id == token.head_token.id:
                        if not sent_id in se["3"]:
                            se["3"][sent_id] = []
                        se["3"][sent_id].append(sentence.map_token_id[token.head_token.id])
                        break
    return se
                
root_se = {}
subordinate_se = {}
for corpus in corpora:
    if corpus in ["bosque"]:
        root_se[corpus] = find_se(corpora[corpus].sentences, root_without_subject[corpus])
        subordinate_se[corpus] = find_se(corpora[corpus].sentences, subordinate_without_subject[corpus])
        
        for filtro in ["1", "2", "3"]:
            for sentence in root_se[corpus][filtro]:
                for token in root_se[corpus][filtro][sentence]:
                    if token in root_without_subject[corpus][sentence]:
                        root_without_subject[corpus][sentence].remove(token)
            if filtro in ["1"]:
                for sentence in subordinate_se[corpus][filtro]:
                    for token in subordinate_se[corpus][filtro][sentence]:
                        if token in subordinate_without_subject[corpus][sentence]:
                            subordinate_without_subject[corpus][sentence].remove(token)

In [ ]:
for corpus in corpora:
    if corpus in ["bosque"]:
        for filtro in ["1", "2", "3"]:
            print("\n{} - orações principais com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in root_se[corpus][filtro].values()])))
            if filtro in ["1"]:
                print("{} - orações subordinadas com sujeito oculto e se (filtro {}): {}".format(corpus, filtro, sum([len(x) for x in subordinate_se[corpus][filtro].values()])))

for corpus in corpora:
    if corpus in ["bosque"]:
        for filtro in ["1", "2", "3"]:
            i = 0
            ii = 0
            print("\n=== {} exemplos de se (filtro {}) como filho de oração principal em {} ===".format(max_examples, filtro, corpus))
            for sent_id, tokens in root_se[corpus][filtro].items():
                i += len(tokens)
                print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
                if i >= max_examples:
                    break
            if filtro in ["1"]:
                print("\n=== {} exemplos de se (filtro {}) como filho de oração subordinada em {} ===".format(max_examples, filtro, corpus))
                for sent_id, tokens in subordinate_se[corpus][filtro].items():
                    ii += len(tokens)
                    print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
                    if ii >= max_examples:
                        break

# Filtro E

In [ ]:
def find_tratarse(sentences, list_tokens):
    tratarse = {}
    for sent_id in list_tokens:
        sentence = sentences[sent_id]
        for t, token in enumerate(sentence.tokens):
            if token.lemma == "tratar" and token.next_token.lemma == "se":
                if not sent_id in tratarse:
                    tratarse[sent_id].append(sentence.map_token_id[token.id])
    return se
                
root_tratarse = {}
subordinate_tratarse = {}
for corpus in corpora:
    if corpus in ["dhbb", "obras"]:
        root_tratarse[corpus] = find_tratarse(corpora[corpus].sentences, root_without_subject[corpus])
        subordinate_tratarse[corpus] = find_tratarse(corpora[corpus].sentences, subordinate_without_subject[corpus])
        
        for sentence in root_tratarse[corpus][filtro]:
            for token in root_tratarse[corpus][filtro][sentence]:
                if token in root_without_subject[corpus][sentence]:
                    root_without_subject[corpus][sentence].remove(token)
        for sentence in subordinate_tratarse[corpus][filtro]:
            for token in subordinate_tratarse[corpus][filtro][sentence]:
                if token in subordinate_without_subject[corpus][sentence]:
                    subordinate_without_subject[corpus][sentence].remove(token)

In [ ]:
for corpus in corpora:
    if corpus in ["dhbb", "obras"]:
        print("\n{} - orações principais com sujeito oculto e tratar-se: {}".format(corpus, sum([len(x) for x in root_tratarse[corpus][filtro].values()])))
        print("{} - orações subordinadas com sujeito oculto e tratar-se: {}".format(corpus, sum([len(x) for x in subordinate_tratarse[corpus][filtro].values()])))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de tratar-se como oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_tratarse[corpus][filtro].items():
        i += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if i >= max_examples:
            break
    print("\n=== {} exemplos de tratar-se como oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_tratarse[corpus][filtro].items():
        ii += len(tokens)
        print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
        if ii >= max_examples:
            break

# Contagem final

In [ ]:
for corpus in corpora:
    print("\n{} - orações principais com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in root_without_subject[corpus].values()]), total_deprel[corpus]["|".join(root_deprel)]))
    print("{} - orações subordinadas com sujeito oculto final: {} / {}".format(corpus, sum([len(x) for x in subordinate_without_subject[corpus].values()]), total_deprel[corpus]["|".join(subordinate_deprel)]))

for corpus in corpora:
    i = 0
    ii = 0
    print("\n=== {} exemplos de sujeito oculto na oração principal em {} ===".format(max_examples, corpus))
    for sent_id, tokens in root_without_subject[corpus].items():
        if len(tokens):
            i += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if i >= max_examples:
                break
    print("\n=== {} exemplos de sujeito oculto na oração subordinada em {} ===".format(max_examples, corpus))
    for sent_id, tokens in subordinate_without_subject[corpus].items():
        if len(tokens):
            ii += len(tokens)
            print(sent_id + ": " + " ".join([x.word if t not in tokens else "*" + x.word + "*" for t, x in enumerate(corpora[corpus].sentences[sent_id].tokens) if not '-' in x.id]))
            if ii >= max_examples:
                break

# Pickle Bosque para posterior reconstituição

In [ ]:
import pickle

for corpus in path:
    for sentence in list(root_without_subject[corpus].keys()):
        if not root_without_subject[corpus][sentence]:
            del root_without_subject[corpus][sentence]
        
    for sentence in list(subordinate_without_subject[corpus].keys()):
        if not subordinate_without_subject[corpus][sentence]:
            del subordinate_without_subject[corpus][sentence]

    if corpus == "bosque":
        with open("{}_root.p".format(corpus), "wb") as f:
            pickle.dump(root_without_subject[corpus], f)
        with open("{}_subordinate.p".format(corpus), "wb") as f:
            pickle.dump(subordinate_without_subject[corpus], f)

# Reconstituição

In [ ]:
import estrutura_ud
import pickle

with open("bosque_root.p", "rb") as f:
    root = pickle.load(f)
with open("bosque_subordinate.p", "rb") as f:
    subordinate = pickle.load(f)
    
len_root = sum([len(root[x]) for x in root])
len_subordinate = sum([len(subordinate[x]) for x in subordinate])
    
bosque = estrutura_ud.Corpus(recursivo=True)
bosque.load("bosque-ud-2.6.conllu")

dic_pronouns = {
    '1Sing': ['eu', 'eu'],
    '1SingMale': ['eu', 'eu'],
    '1SingFem': ['eu', 'eu'],
    '2Sing': ['tu', 'tu'],
    '2SingMale': ['tu', 'tu'],
    '2SingFem': ['tu', 'tu'],
    '3SingMale': ['ele', 'ele'],
    '3SingFem': ['ela', 'ela'],
    '1Plur': ['nós', 'nós'],
    '1PlurMale': ['nós', 'nós'],
    '1PlurFem': ['nós', 'nós'],
    '2Plur': ['vós', 'vós'],
    '2PlurMale': ['vós', 'vós'],
    '2PlurFem': ['vós', 'vós'],
    '3PlurMale': ['eles', 'eles'],
    '3PlurFem': ['elas', 'elas']
}

bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].deprel = "advcl"
bosque.sentences["CP493-7"].tokens[bosque.sentences["CP493-7"].map_token_id["26"]].dephead = "13"            

In [ ]:
def find_feats(feat, token):
    if not feat in token.feats:
        return ""
    return token.feats.split(feat + "=")[1].split("|")[0]

In [ ]:
def insert_phrase(sentence, first_token_id, phrase):
    sintagmas_anteriores = []
    for t, token in enumerate(sentence.tokens):
        if t < sentence.map_token_id[first_token_id]:
            if token.dephead == first_token_id and (token.upos == "ADV" or (token.lemma == "se" and token.upos == "PRON") or token.deprel == "cop" or token.upos == "AUX"):
                is_sintagma = True
                for _token in sentence.tokens:
                    if _token.upos == "PUNCT" and _token.dephead == token.id:
                        is_sintagma = False
                        break
                if is_sintagma:
                    sintagmas_anteriores.append(sentence.tokens[t].id)
    who_head = first_token_id
    first_token_id = min(sintagmas_anteriores) if sintagmas_anteriores else first_token_id

    for new_token in reversed(phrase):
        token = estrutura_ud.Token()
        token.build(new_token.to_str())
        token.dephead = str(int(token.dephead) - int(token.id) + int(who_head.split("-")[0]))
        token.id = first_token_id.split("-")[0]
        if first_token_id == "1":
            if token.word != "<SUBJ>":
                token.word = token.word.title()
            sentence.tokens[0].word = sentence.tokens[0].word.lower()
        token.misc = "|".join(sorted([x for x in token.misc.split("|") + ["<SUBJ>"] if x != "_"]))
        sentence.tokens.insert(sentence.map_token_id[first_token_id], token)
        
        t_first_token_id = sentence.map_token_id[first_token_id]
        for t, token in enumerate(sentence.tokens):
            if t > t_first_token_id:
                token.id = str(int(token.id)+1) if not '-' in token.id else str(int(token.id.split("-")[0])+1) + "-" + str(int(token.id.split("-")[1])+1)
            sentence.map_token_id[token.id] = t
        for t, token in enumerate(sentence.tokens):
            if token.dephead not in ["0", "_"] and sentence.map_token_id[token.dephead] >= sentence.map_token_id[first_token_id]:
                token.dephead = str(int(token.dephead)+1)
        
    sentence.refresh_map_token_id()
                      
    sentence.metadados['text'] = " ".join([x.word for x in sentence.tokens if not '-' in x.id])
    return sentence

In [ ]:
#ADVCL à esquerda com mesmo PESSNUM (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del root[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_root))
print("\n" + "\n\n".join(reconstituidos[:20]))

In [ ]:
#ADVCL à esquerda com mesmo PESSNUM (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    sentence_qualifies = False
    to_add = {}
    for t in tokens:
        for _t, _token in enumerate(sentence.tokens):
            if _token.deprel == "advcl" and sentence.tokens[t].id == _token.dephead and _t < t and find_feats("Person", _token) == find_feats("Person", sentence.tokens[t]) and find_feats("Number", _token) == find_feats("Number", sentence.tokens[t]):
                for __token in sentence.tokens:
                    if __token.deprel in ["nsubj", "csubj", "nsubj:pass"] and __token.dephead == _token.id:
                        sentence_qualifies = True
                        to_add[t] = []
                        for ___token in sentence.tokens:
                            if ___token.dephead == __token.id or ___token.id == __token.id:
                                to_add[t].append(___token)
                        break
    
    if sentence_qualifies:
        for i, t in enumerate(to_add):
            bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), to_add[t])
        del subordinate[sent_id]
        reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_subordinate))
print("\n" + "\n\n".join(reconstituidos[:20]))

In [ ]:
#DESINÊNCIA (root)
reconstituidos = []
for sent_id, tokens in list(root.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for i, t in enumerate(tokens):
        has_aux = False
        for _t, _token in enumerate(sentence.tokens):
            if _token.dephead == sentence.tokens[t].id and _token.upos == "AUX":
                person = find_feats("Person", sentence.tokens[_t])
                number = find_feats("Number", sentence.tokens[_t])
                gender = find_feats("Gender", sentence.tokens[_t])
                has_aux = True
                break
        if not has_aux:
            person = find_feats("Person", sentence.tokens[t])
            number = find_feats("Number", sentence.tokens[t])
            gender = find_feats("Gender", sentence.tokens[t])
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'PRON', '_', feats, "1", 'nsubj', '_', '_']
        else:
            pronoun = ["1", "<SUBJ>", "<SUBJ>", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun))
    
        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], str(int(bosque.sentences[sent_id].tokens[t+i].id)), [new_token])
    del root[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_root))
print("\n" + "\n\n".join(reconstituidos[:20]))

In [ ]:
#DESINÊNCIA (subordinate)
reconstituidos = []
for sent_id, tokens in list(subordinate.items()):
    sentence = bosque.sentences[sent_id]
    to_add = {}
    for i, t in enumerate(tokens):
        has_aux = False
        for _t, _token in enumerate(sentence.tokens):
            if _token.dephead == sentence.tokens[t].id and _token.upos == "AUX":
                person = find_feats("Person", sentence.tokens[_t])
                number = find_feats("Number", sentence.tokens[_t])
                gender = find_feats("Gender", sentence.tokens[_t])
                has_aux = True
                break
        if not has_aux:
            person = find_feats("Person", sentence.tokens[t])
            number = find_feats("Number", sentence.tokens[t])
            gender = find_feats("Gender", sentence.tokens[t])
        feats = []
        if person:
            feats.append("Person=" + person)
        if number:
            feats.append("Number=" + number)
        if gender:
            feats.append("Gender=" + gender)
        feats = "|".join(sorted(feats)) if feats else "_"
        pronoun = person + number + gender
        if pronoun in dic_pronouns:
            pronoun = ["1", dic_pronouns[pronoun][0], dic_pronouns[pronoun][1], 'PRON', '_', feats, "1", 'nsubj', '_', '_']
        else:
            pronoun = ["1", "<SUBJ>", "<SUBJ>", "NOUN", "_", feats, "1", "nsubj", "_", "_"]
        new_token = estrutura_ud.Token()
        new_token.build("\t".join(pronoun)) 

        bosque.sentences[sent_id] = insert_phrase(bosque.sentences[sent_id], bosque.sentences[sent_id].tokens[t+i].id, [new_token])
    del subordinate[sent_id]
    reconstituidos.append(sent_id + ":\n" + bosque.sentences[sent_id].text + "\n" + bosque.sentences[sent_id].metadados['text'])

print("Reconstituídos: {} / {}".format(len(reconstituidos), len_subordinate))
print("\n" + "\n\n".join(reconstituidos[:20]))

In [ ]:
# Salvar modificado
bosque.save("bosque-ud-2.6-desocultado.conllu")

# Avaliação

# Debugagem

In [ ]:
print(corpora['obras'].sentences["540"].to_str())

In [ ]:
#root_nominals["obras"]
root_without_subject["obras"]